# Биоинформатический анализ генома: поиск и предсказание генов

Этот notebook демонстрирует основные методы работы с геномными данными:
1. Загрузка и парсинг геномных файлов (FASTA, GFF)
2. Поиск открытых рамок считывания (ORF)
3. Сравнение предсказанных генов с референсными данными
4. Оценка качества предсказания

## ЧАСТЬ 1: ЗАГРУЗКА ДАННЫХ E.COLI

Загружаем референсный геном кишечной палочки E.coli strain K-12 - один из наиболее изученных бактериальных геномов.

In [ ]:
print("=== Загрузка генома E.coli ===")
print("Скачиваем референсный геном кишечной палочки E.coli strain K-12")
print("Это один из наиболее изученных бактериальных геномов")

# Загружаем сжатый геном E.coli
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz

In [ ]:
# Проверяем размер скачанного файла
!ls -hlrt

print("\nРаспаковываем геном...")
# Распаковываем архив
!gzip -d GCF_000005845.2_ASM584v2_genomic.fna.gz

# Проверяем размер распакованного файла
!ls -hlrt

In [ ]:
print("\nПросматриваем начало файла:")
print("FASTA формат содержит:")
print("- Заголовок, начинающийся с '>'")
print("- Последовательность ДНК в стандартных символах (A, T, G, C)")

# Смотрим первые 10 строк генома
!head GCF_000005845.2_ASM584v2_genomic.fna

## ЧАСТЬ 2: ПАРСИНГ FASTA ФАЙЛА

Создаем функцию для чтения FASTA формата и загружаем геномную последовательность.

In [ ]:
print("\n=== Парсинг FASTA файла ===")
print("Создаем функцию для чтения FASTA формата")

def parse_fasta(filepath):
    """
    Парсит FASTA файл и возвращает пары (заголовок, последовательность)
    
    FASTA формат:
    >Заголовок_последовательности
    ATGCGATCGATCG...
    ATCGATCGATCGA...
    
    Args:
        filepath: путь к FASTA файлу
    
    Yields:
        tuple: (заголовок, последовательность)
    """
    header = None
    sequence_lines = []

    with open(filepath, 'r') as file:
        for line in file:
            line = line.strip()  # Удаляем пробелы и символы новой строки
            if not line:
                continue  # Пропускаем пустые строки
            if line.startswith('>'):
                if header is not None:
                    # Если уже есть заголовок, возвращаем предыдущую последовательность
                    yield header, ''.join(sequence_lines)
                header = line[1:]  # Убираем '>' в начале
                sequence_lines = []  # Сброс буфера для новой последовательности
            else:
                sequence_lines.append(line)

        # Возвращаем последнюю последовательность в файле
        if header is not None:
            yield header, ''.join(sequence_lines)

In [ ]:
# Применяем парсер к нашему файлу
fasta_path = "GCF_000005845.2_ASM584v2_genomic.fna"

print("Анализируем структуру FASTA файла:")
for header, sequence in parse_fasta(fasta_path):
    print("Header:", header)
    print("Sequence length:", len(sequence), "nucleotides")
    print("First 60 characters:", sequence[:60] + '...')
    
    # Сохраняем последовательность для дальнейшего анализа
    genome_sequence = sequence
    break  # E.coli имеет одну хромосому, поэтому берем первую

print(f"\nПолная длина генома E.coli: {len(genome_sequence):,} нуклеотидов")

## ЧАСТЬ 3: БАЗОВЫЕ ОПЕРАЦИИ С ДНК

ДНК двуцепочечная, и гены могут находиться на любой из цепей. Для анализа противоположной цепи нужно вычислить обратную комплементарную последовательность.

In [ ]:
print("\n=== Базовые операции с ДНК ===")

def reverse_complement(dna_seq):
    """
    Вычисляет обратную комплементарную последовательность ДНК
    
    ДНК двуцепочечная, и гены могут находиться на любой из цепей.
    Для анализа противоположной цепи нужно:
    1. Заменить каждый нуклеотид на комплементарный (A↔T, G↔C)
    2. Обратить последовательность (читать справа налево)
    
    Args:
        dna_seq: последовательность ДНК
    
    Returns:
        str: обратная комплементарная последовательность
    """
    complement = {
        'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G',
        'a': 't', 't': 'a', 'g': 'c', 'c': 'g',
        'N': 'N', 'n': 'n'  # N обозначает неопределенный нуклеотид
    }
    reversed_seq = dna_seq[::-1]  # Обращаем последовательность
    rev_comp = ''.join(complement.get(base, base) for base in reversed_seq)
    return rev_comp

# Демонстрируем работу функции
test_seq = "ATGCGATCG"
print(f"Исходная последовательность: {test_seq}")
print(f"Обратная комплементарная:    {reverse_complement(test_seq)}")

print(f"\nДлина генома: {len(genome_sequence):,} п.н.")

# Создаем обратную комплементарную последовательность для анализа минус-цепи
rev_comp_sequence = reverse_complement(genome_sequence)
print("Обратная комплементарная последовательность создана")

## ЧАСТЬ 4: ПОИСК СТАРТОВЫХ КОДОНОВ

В бактериях гены обычно начинаются со стартового кодона ATG (метионин). Анализируем их распределение.

In [ ]:
print("\n=== Анализ стартовых кодонов ===")
print("В бактериях гены обычно начинаются со стартового кодона ATG (метионин)")

# Подсчитываем стартовые кодоны на обеих цепях
methionine_start = "ATG"
methionine_start_rc = "CAT"  # ATG на обратной цепи выглядит как CAT на прямой

hits_plus = genome_sequence.count(methionine_start)
hits_minus = genome_sequence.count(methionine_start_rc)

print(f"Стартовые кодоны ATG на плюс-цепи:  {hits_plus:,}")
print(f"Стартовые кодоны ATG на минус-цепи: {hits_minus:,}")
print(f"Общее количество ATG:              {hits_plus + hits_minus:,}")

print(f"\nЧастота ATG: {(hits_plus + hits_minus) / len(genome_sequence) * 1000:.2f} на 1000 п.н.")
print("Это примерно соответствует случайному распределению (ожидаемая частота ATG = 1/64 ≈ 0.016)")

## ЧАСТЬ 5: ЗАГРУЗКА АННОТАЦИИ ГЕНОВ (GFF)

GFF (General Feature Format) содержит координаты и описания генов - эталонные данные для сравнения.

In [ ]:
print("\n=== Загрузка аннотации генов ===")
print("GFF (General Feature Format) содержит координаты и описания генов")

# Загружаем GFF файл с аннотацией E.coli
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gff.gz

# Распаковываем
!gzip -d GCF_000005845.2_ASM584v2_genomic.gff.gz

In [ ]:
print("\nПросматриваем структуру GFF файла:")
print("GFF содержит 9 колонок:")
print("1. Хромосома/контиг")
print("2. Источник аннотации") 
print("3. Тип элемента (gene, CDS, etc.)")
print("4. Начало (1-based)")
print("5. Конец (включительно)")
print("6. Скор")
print("7. Цепь (+/-)")
print("8. Фаза")
print("9. Атрибуты")

# Показываем первые строки GFF
!head GCF_000005845.2_ASM584v2_genomic.gff

## ЧАСТЬ 6: ПАРСИНГ GFF ФАЙЛА

Создаем функцию для извлечения информации о генах из GFF файла.

In [ ]:
print("\n=== Парсинг GFF файла ===")

def parse_gff(filepath):
    """
    Парсит GFF файл и извлекает информацию о генетических элементах
    
    Args:
        filepath: путь к GFF файлу
    
    Returns:
        list: список словарей с информацией о каждом элементе
    """
    annotations = []
    with open(filepath, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith('#'):
                continue  # Пропускаем комментарии и пустые строки

            parts = line.split('\t')
            if len(parts) != 9:
                continue  # Пропускаем некорректные строки

            seqid, source, feature_type, start, end, score, strand, phase, attributes = parts

            # Парсим колонку attributes в словарь
            attr_dict = {}
            for attr in attributes.split(';'):
                if '=' in attr:
                    key, value = attr.split('=', 1)
                    attr_dict[key] = value

            annotations.append({
                'seqid': seqid,
                'source': source,
                'type': feature_type,
                'start': int(start),
                'end': int(end),
                'score': score if score != '.' else None,
                'strand': strand,
                'phase': phase if phase != '.' else None,
                'attributes': attr_dict
            })

    return annotations

# Парсим GFF файл
print("Парсинг GFF файла...")
annotations = parse_gff("GCF_000005845.2_ASM584v2_genomic.gff")
print(f"Загружено {len(annotations)} аннотационных записей")

# Анализируем типы элементов
from collections import Counter
feature_types = Counter([ann['type'] for ann in annotations])
print("\nТипы генетических элементов:")
for feature_type, count in feature_types.most_common():
    print(f"  {feature_type}: {count}")

## ЧАСТЬ 7: АНАЛИЗ ДЛИН ГЕНОВ

Строим распределение длин генов и находим экстремальные значения.

In [ ]:
print("\n=== Анализ длин генов ===")

import matplotlib.pyplot as plt

def plot_gene_lengths(gff_file):
    """
    Строит гистограмму распределения длин генов и находит экстремальные значения
    """
    features = parse_gff(gff_file)
    genes = [f for f in features if f['type'] == 'gene']
    gene_lengths = [gene['end'] - gene['start'] + 1 for gene in genes]

    if not gene_lengths:
        print("Гены не найдены в GFF файле")
        return

    # Находим самый короткий и самый длинный ген
    min_length = min(gene_lengths)
    max_length = max(gene_lengths)
    avg_length = sum(gene_lengths) / len(gene_lengths)

    min_index = gene_lengths.index(min_length)
    max_index = gene_lengths.index(max_length)

    # Строим гистограмму
    plt.figure(figsize=(12, 6))
    plt.hist(gene_lengths, bins=50, edgecolor='black', alpha=0.7)
    plt.title("Распределение длин генов E.coli")
    plt.xlabel("Длина гена (нуклеотиды)")
    plt.ylabel("Количество генов")
    plt.axvline(avg_length, color='red', linestyle='--', label=f'Средняя длина: {avg_length:.0f} п.н.')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Выводим статистику
    print(f"Общее количество генов: {len(genes)}")
    print(f"Самый короткий ген: {min_length} п.н.")
    print(f"Самый длинный ген: {max_length} п.н.")
    print(f"Средняя длина гена: {avg_length:.1f} п.н.")
    
    # Информация о экстремальных генах
    min_gene = genes[min_index]
    max_gene = genes[max_index]
    
    print(f"\nСамый короткий ген:")
    print(f"  ID: {min_gene['attributes'].get('ID', 'неизвестно')}")
    print(f"  Позиция: {min_gene['start']}-{min_gene['end']}")
    
    print(f"\nСамый длинный ген:")
    print(f"  ID: {max_gene['attributes'].get('ID', 'неизвестно')}")
    print(f"  Позиция: {max_gene['start']}-{max_gene['end']}")

# Анализируем длины генов E.coli
gff_path = "GCF_000005845.2_ASM584v2_genomic.gff"
plot_gene_lengths(gff_path)

## ЧАСТЬ 8: ПОИСК ОТКРЫТЫХ РАМОК СЧИТЫВАНИЯ (ORF)

ОРФ (Open Reading Frame) - участок ДНК, который потенциально может кодировать белок.

**Критерии ОРФ:**
1. Начинается со стартового кодона (ATG)
2. Заканчивается стоп-кодоном (TAA, TAG, TGA)
3. Находится в одной рамке считывания (кратно 3 нуклеотидам)
4. Не содержит промежуточных стоп-кодонов

**У ДНК есть 6 возможных рамок считывания:**
- 3 на прямой цепи (начиная с позиций 0, 1, 2)
- 3 на обратной цепи

In [ ]:
def find_orfs_bacterial(dna_sequence: str) -> list:
    """
    Находит открытые рамки считывания (ОРФ) в геномной последовательности.
    
    Алгоритм:
    1. Ищем все ATG кодоны в каждой из 3 рамок считывания
    2. Для каждого ATG ищем ближайший стоп-кодон в той же рамке
    3. Проверяем, что ОРФ не вложен в уже найденный ОРФ той же рамки
    
    Args:
        dna_sequence: последовательность ДНК
    
    Returns:
        list: список словарей с информацией об ОРФах
    """
    if not dna_sequence:
        return []

    dna_sequence = dna_sequence.upper()
    n = len(dna_sequence)

    start_codon = "ATG"
    stop_codons = ["TAA", "TAG", "TGA"]

    found_orfs = []
    
    # Для каждой рамки отслеживаем найденные ОРФы
    identified_orf_regions_by_frame = {0: [], 1: [], 2: []}

    # Анализируем 3 рамки считывания
    for frame_offset in range(3):
        print(f"  Анализируем рамку {frame_offset + 1}...")
        
        # Находим все ATG в текущей рамке
        potential_atg_indices = []
        for i in range(frame_offset, n - 2, 3):
            codon = dna_sequence[i : i + 3]
            if codon == start_codon:
                potential_atg_indices.append(i)

        print(f"    Найдено {len(potential_atg_indices)} потенциальных стартовых кодонов")

        # Для каждого ATG ищем ОРФ
        valid_orfs_in_frame = 0
        for atg_start_index in potential_atg_indices:
            # Проверяем, не вложен ли этот ATG в уже найденный ОРФ
            is_nested = False
            for orf_start, orf_end in identified_orf_regions_by_frame[frame_offset]:
                if orf_start <= atg_start_index < orf_end:
                    is_nested = True
                    break

            if is_nested:
                continue

            # Ищем первый стоп-кодон после ATG
            for j in range(atg_start_index + 3, n - 2, 3):
                codon = dna_sequence[j : j + 3]
                if codon in stop_codons:
                    orf_end_index = j + 3
                    orf_sequence = dna_sequence[atg_start_index:orf_end_index]
                    
                    orf_info = {
                        "start": atg_start_index,
                        "end": orf_end_index,
                        "frame": frame_offset + 1,
                        "sequence": orf_sequence,
                        "length": len(orf_sequence)
                    }
                    found_orfs.append(orf_info)
                    
                    # Запоминаем регион этого ОРФа
                    identified_orf_regions_by_frame[frame_offset].append(
                        (atg_start_index, orf_end_index)
                    )
                    valid_orfs_in_frame += 1
                    break

        print(f"    Найдено {valid_orfs_in_frame} валидных ОРФов")

    # Сортируем по позиции
    found_orfs.sort(key=lambda x: (x['start'], x['frame']))
    return found_orfs

In [ ]:
# Поиск ОРФов на обеих цепях
print("Поиск ОРФов на прямой цепи:")
orfs_plus = find_orfs_bacterial(genome_sequence)

print("\nПоиск ОРФов на обратной цепи:")
orfs_minus = find_orfs_bacterial(reverse_complement(genome_sequence))

print(f"\nРезультаты поиска ОРФов:")
print(f"Прямая цепь:   {len(orfs_plus):,} ОРФов")
print(f"Обратная цепь: {len(orfs_minus):,} ОРФов")
print(f"Всего:         {len(orfs_plus) + len(orfs_minus):,} ОРФов")

# Показываем примеры найденных ОРФов
print("\nПримеры найденных ОРФов (первые 5):")
for i, orf in enumerate(orfs_plus[:5]):
    print(f"ОРФ {i+1}: позиция {orf['start']}-{orf['end']}, "
          f"рамка {orf['frame']}, длина {orf['length']} п.н.")
    print(f"  Последовательность: {orf['sequence'][:60]}...")

## ЧАСТЬ 9: АНАЛИЗ РАСПРЕДЕЛЕНИЯ ДЛИН ОРФов

Строим гистограммы распределения длин найденных ОРФов.

In [ ]:
print("\n=== Анализ распределения длин ОРФов ===")

def plot_orf_length_distribution(orfs_list, title="Распределение длин ОРФов"):
    """
    Строит гистограмму распределения длин ОРФов
    """
    if not orfs_list:
        print(f"ОРФы не найдены для: {title}")
        return

    orf_lengths = [orf['length'] for orf in orfs_list]

    plt.figure(figsize=(12, 6))
    plt.hist(orf_lengths, bins=50, edgecolor='black', alpha=0.7)
    plt.title(title)
    plt.xlabel("Длина ОРФа (нуклеотиды)")
    plt.ylabel("Количество ОРФов")
    plt.yscale('log')  # Логарифмическая шкала для лучшей визуализации
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Статистика
    min_length = min(orf_lengths)
    max_length = max(orf_lengths)
    avg_length = sum(orf_lengths) / len(orf_lengths)
    
    print(f"Общее количество ОРФов: {len(orf_lengths):,}")
    print(f"Самый короткий ОРФ: {min_length} п.н.")
    print(f"Самый длинный ОРФ: {max_length} п.н.")
    print(f"Средняя длина ОРФа: {avg_length:.1f} п.н.")

# Анализируем распределение длин
plot_orf_length_distribution(orfs_plus, "Распределение длин ОРФов (прямая цепь)")
plot_orf_length_distribution(orfs_minus, "Распределение длин ОРФов (обратная цепь)")

## ЧАСТЬ 10: ИЗВЛЕЧЕНИЕ РЕФЕРЕНСНЫХ ГЕНОВ ИЗ GFF

Извлекаем последовательности реальных генов для сравнения с нашими предсказаниями.

In [ ]:
print("\n=== Извлечение референсных генов из GFF ===")

# Загружаем BioPython если его нет
try:
    from Bio import SeqIO
except ImportError:
    print("Устанавливаем BioPython...")
    !pip install biopython
    from Bio import SeqIO

import csv

def extract_orfs_from_gff(gff_file, fasta_file):
    """
    Извлекает последовательности генов из GFF аннотации и FASTA файла
    """
    # Загружаем последовательности
    seq_dict = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
    orfs = []

    with open(gff_file, 'r') as f:
        reader = csv.reader(f, delimiter='\t')
        for row in reader:
            if len(row) < 9 or row[0].startswith('#') or row[2] != 'gene':
                continue
                
            chrom = row[0]
            start = int(row[3]) - 1  # Конвертируем в 0-based координаты
            end = int(row[4])
            strand = row[6]

            # Извлекаем последовательность
            sequence = seq_dict[chrom].seq[start:end]
            if strand == '-':
                continue  # Пока анализируем только прямую цепь
                
            orfs.append({
                'start': start,
                'end': end,
                'frame': (start) % 3,  # Упрощенное определение рамки
                'sequence': str(sequence),
                'length': end - start
            })

    return orfs

# Извлекаем референсные гены
print("Извлечение референсных генов из аннотации...")
gff_file = "GCF_000005845.2_ASM584v2_genomic.gff"
fasta_file = "GCF_000005845.2_ASM584v2_genomic.fna"
reference_orfs = extract_orfs_from_gff(gff_file, fasta_file)

print(f"Извлечено {len(reference_orfs)} референсных генов")
print(f"Предсказано {len(orfs_plus)} ОРФов")

# Показываем пример референсного гена
if reference_orfs:
    print(f"\nПример референсного гена:")
    ref_gene = reference_orfs[0]
    print(f"Позиция: {ref_gene['start']}-{ref_gene['end']}")
    print(f"Длина: {ref_gene['length']} п.н.")
    print(f"Последовательность: {ref_gene['sequence'][:60]}...")

## ЧАСТЬ 11: ОЦЕНКА КАЧЕСТВА ПРЕДСКАЗАНИЯ

Для оценки качества предсказания используем стандартные метрики:

- **True Positive (TP)** - правильно предсказанные гены
- **False Positive (FP)** - ложно предсказанные гены (предсказали, но гена нет)
- **False Negative (FN)** - пропущенные гены (ген есть, но не предсказали)

**Метрики:**
- **Precision (точность)** = TP / (TP + FP) - доля правильных среди предсказанных
- **Recall (полнота)** = TP / (TP + FN) - доля найденных среди всех реальных
- **F1-score** = 2 * (Precision * Recall) / (Precision + Recall) - гармоническое среднее

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

def match_orfs(predicted, truth, tolerance=0):
    """
    Сопоставляет предсказанные ОРФы с референсными
    
    Args:
        predicted: список предсказанных ОРФов
        truth: список референсных ОРФов
        tolerance: допустимое отклонение в координатах
    
    Returns:
        tuple: (TP, FP, FN)
    """
    pred_intervals = [(orf['start'], orf['end']) for orf in predicted]
    true_intervals = [(orf['start'], orf['end']) for orf in truth]
    
    matched_true = set()
    tp = 0  # True Positives
    fp = 0  # False Positives

    # Проверяем каждый предсказанный ОРФ
    for p_start, p_end in pred_intervals:
        found_match = False
        for idx, (t_start, t_end) in enumerate(true_intervals):
            if idx in matched_true:
                continue
            
            # Проверяем совпадение координат с допуском
            if (abs(p_start - t_start) <= tolerance and 
                abs(p_end - t_end) <= tolerance):
                tp += 1
                matched_true.add(idx)
                found_match = True
                break
        
        if not found_match:
            fp += 1

    fn = len(true_intervals) - len(matched_true)  # False Negatives
    return tp, fp, fn

def compute_metrics(tp, fp, fn):
    """Вычисляет метрики качества"""
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
    accuracy = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0.0
    
    return {
        'TP': tp,
        'FP': fp,
        'FN': fn,
        'Accuracy': round(accuracy, 4),
        'Precision': round(precision, 4),
        'Recall': round(recall, 4),
        'F1 Score': round(f1, 4),
    }

def evaluate_prediction(predicted_orfs, reference_orfs, tolerance=0):
    """Оценивает качество предсказания и выводит результаты"""
    tp, fp, fn = match_orfs(predicted_orfs, reference_orfs, tolerance)
    metrics = compute_metrics(tp, fp, fn)
    
    print(f"Результаты оценки (допуск ±{tolerance} п.н.):")
    print("-" * 50)
    for metric, value in metrics.items():
        print(f"{metric:12}: {value}")
    
    print(f"\nИнтерпретация:")
    print(f"- Из {len(reference_orfs)} реальных генов найдено {tp} ({metrics['Recall']*100:.1f}%)")
    print(f"- Из {len(predicted_orfs)} предсказанных {tp} правильные ({metrics['Precision']*100:.1f}%)")
    print(f"- Пропущено {fn} генов")
    print(f"- Ложно предсказано {fp} генов")
    
    return metrics

# Оценка базового предсказания
print("Оценка базового алгоритма поиска ОРФов:")
metrics_base = evaluate_prediction(orfs_plus, reference_orfs, tolerance=0)

## ЧАСТЬ 12: ФИЛЬТРАЦИЯ ОРФов ПО ДЛИНЕ

Большинство найденных ОРФов очень короткие и вряд ли кодируют белки. Применим фильтр по минимальной длине.

In [ ]:
print("\n=== Фильтрация ОРФов по длине ===")
print("Большинство найденных ОРФов очень короткие и вряд ли кодируют белки")
print("Применим фильтр: оставим только ОРФы длиннее 150 нуклеотидов (50 аминокислот)")

# Фильтруем короткие ОРФы
min_length = 150  # нуклеотидов
orfs_filtered = [orf for orf in orfs_plus if orf["length"] >= min_length]

print(f"До фильтрации: {len(orfs_plus):,} ОРФов")
print(f"После фильтрации: {len(orfs_filtered):,} ОРФов")
print(f"Отфильтровано: {len(orfs_plus) - len(orfs_filtered):,} коротких ОРФов")

# Оценка после фильтрации
print("\nОценка после фильтрации по длине:")
metrics_filtered = evaluate_prediction(orfs_filtered, reference_orfs, tolerance=0)

# Сравнение результатов
print(f"\nСравнение результатов:")
print(f"{'Метрика':<12} {'Базовый':<10} {'Фильтрация':<12} {'Изменение'}")
print("-" * 50)
for metric in ['Precision', 'Recall', 'F1 Score']:
    base_val = metrics_base[metric]
    filt_val = metrics_filtered[metric]
    change = filt_val - base_val
    change_str = f"{change:+.3f}"
    print(f"{metric:<12} {base_val:<10.3f} {filt_val:<12.3f} {change_str}")

## ЧАСТЬ 13: АНАЛИЗ ОШИБОК ПРЕДСКАЗАНИЯ

Подробный анализ того, где наш алгоритм ошибается.

In [ ]:
print("\n=== Анализ ошибок предсказания ===")

def analyze_prediction_errors(predicted, reference, tolerance=0):
    """
    Подробный анализ того, где наш алгоритм ошибается
    """
    # Классифицируем каждый предсказанный ОРФ
    matched_ref = set()
    classified_pred = []
    
    for pred_orf in predicted:
        match_type = 'FP'  # По умолчанию False Positive
        
        for idx, ref_orf in enumerate(reference):
            if idx in matched_ref:
                continue
                
            # Проверяем разные типы совпадений
            start_match = abs(pred_orf['start'] - ref_orf['start']) <= tolerance
            end_match = abs(pred_orf['end'] - ref_orf['end']) <= tolerance
            
            if start_match and end_match:
                match_type = 'TP-full'  # Полное совпадение
                matched_ref.add(idx)
                break
            elif start_match:
                match_type = 'TP-start'  # Совпадает только начало
                matched_ref.add(idx)
                break
            elif end_match:
                match_type = 'TP-end'  # Совпадает только конец
                matched_ref.add(idx)
                break
        
        classified_pred.append({**pred_orf, 'type': match_type})
    
    # Классифицируем референсные ОРФы
    classified_ref = []
    for idx, ref_orf in enumerate(reference):
        ref_type = 'FN' if idx not in matched_ref else 'TP'
        classified_ref.append({**ref_orf, 'type': ref_type})
    
    return classified_pred, classified_ref

# Анализируем ошибки
print("Классификация предсказанных ОРФов...")
pred_classified, ref_classified = analyze_prediction_errors(
    orfs_filtered, reference_orfs, tolerance=0
)

# Подсчитываем типы совпадений
from collections import Counter
pred_types = Counter([orf['type'] for orf in pred_classified])

print("Результаты классификации:")
print(f"  Полные совпадения (TP-full):  {pred_types['TP-full']}")
print(f"  Совпадения по началу:         {pred_types['TP-start']}")
print(f"  Совпадения по концу:          {pred_types['TP-end']}")
print(f"  Ложные предсказания (FP):     {pred_types['FP']}")

fn_count = len([orf for orf in ref_classified if orf['type'] == 'FN'])
print(f"  Пропущенные гены (FN):        {fn_count}")

## ЧАСТЬ 14: АНАЛИЗ GC-СОСТАВА

GC-состав может помочь отличить настоящие гены от случайных ОРФов.

In [ ]:
print("\n=== Анализ GC-состава ===")
print("GC-состав может помочь отличить настоящие гены от случайных ОРФов")

def gc_content(sequence):
    """Вычисляет GC-состав последовательности (в процентах)"""
    sequence = sequence.upper()
    gc_count = sequence.count('G') + sequence.count('C')
    return 100 * gc_count / len(sequence) if len(sequence) > 0 else 0

def compare_gc_content(predicted_classified):
    """Сравнивает GC-состав правильных и ложных предсказаний"""
    tp_gc = []
    fp_gc = []
    
    for orf in predicted_classified:
        gc = gc_content(orf['sequence'])
        if orf['type'] == 'TP-full':
            tp_gc.append(gc)
        elif orf['type'] == 'FP':
            fp_gc.append(gc)
    
    if tp_gc and fp_gc:
        avg_tp = sum(tp_gc) / len(tp_gc)
        avg_fp = sum(fp_gc) / len(fp_gc)
        
        print(f"Средний GC-состав:")
        print(f"  Правильные предсказания: {avg_tp:.1f}% (n={len(tp_gc)})")
        print(f"  Ложные предсказания:     {avg_fp:.1f}% (n={len(fp_gc)})")
        
        # Строим гистограмму
        plt.figure(figsize=(10, 6))
        plt.hist(tp_gc, bins=20, alpha=0.7, label='Правильные (TP)', density=True)
        plt.hist(fp_gc, bins=20, alpha=0.7, label='Ложные (FP)', density=True)
        plt.xlabel('GC-состав (%)')
        plt.ylabel('Плотность')
        plt.title('Распределение GC-состава: правильные vs ложные предсказания')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        return avg_tp, avg_fp
    else:
        print("Недостаточно данных для анализа GC-состава")
        return None, None

# Анализируем GC-состав
tp_gc, fp_gc = compare_gc_content(pred_classified)

## ЗАКЛЮЧЕНИЕ

Результаты анализа показывают важность комплексного подхода к предсказанию генов и необходимость использования множественных критериев для отличия настоящих генов от случайных открытых рамок считывания.

### Основные выводы:
1. **Высокий уровень ложных предсказаний** - большинство ОРФов не являются генами
2. **Простой поиск по стартовым/стоп-кодонам недостаточен**
3. **Нужны дополнительные фильтры** (GC-состав, Shine-Dalgarno, кодонное смещение)

### Возможные улучшения:
1. Поиск Shine-Dalgarno последовательностей перед ATG
2. Анализ кодонного смещения
3. Машинное обучение на признаках последовательностей
4. Сравнительная геномика
5. Анализ экспрессии (RNA-seq данные)

In [ ]:
print("\n" + "="*60)
print("ЗАКЛЮЧЕНИЕ")
print("="*60)

print(f"""
Результаты анализа генома E.coli:

📊 СТАТИСТИКА ГЕНОМА:
  • Длина генома: {len(genome_sequence):,} нуклеотидов
  • Референсных генов: {len(reference_orfs)}
  • Найдено потенциальных ОРФов: {len(orfs_plus):,}
  • После фильтрации по длине: {len(orfs_filtered):,}

🎯 КАЧЕСТВО ПРЕДСКАЗАНИЯ:
  • Точность (Precision): {metrics_filtered['Precision']:.1%}
  • Полнота (Recall): {metrics_filtered['Recall']:.1%}
  • F1-мера: {metrics_filtered['F1 Score']:.3f}

🔍 ОСНОВНЫЕ ПРОБЛЕМЫ:
  1. Высокий уровень ложных предсказаний - большинство ОРФов не являются генами
  2. Простой поиск по стартовым/стоп-кодонам недостаточен
  3. Нужны дополнительные фильтры (GC-состав, Shine-Dalgarno, кодонное смещение)

💡 ВОЗМОЖНЫЕ УЛУЧШЕНИЯ:
  1. Поиск Shine-Dalgarno последовательностей перед ATG
  2. Анализ кодонного смещения
  3. Машинное обучение на признаках последовательностей
  4. Сравнительная геномика
  5. Анализ экспрессии (RNA-seq данные)

Этот анализ показывает важность комплексного подхода к предсказанию генов
и необходимость использования множественных критериев для отличия 
настоящих генов от случайных открытых рамок считывания.
""")

print("\nНастройте параметры фильтрации и попробуйте другие подходы!")
print("Удачи в изучении биоинформатики! 🧬")